# Setting up JupyerHub Dev Env (TLJH)

## Setting up TLJH on Google Cloud

create VM instance
- use ubuntu image
- select "no service account"
- allow both HTTP and HTTPS traffic
- add a startup script with the following
   
```
#!/bin/bash
curl -L https://tljh.jupyter.org/bootstrap.py \
  | sudo python3 - \
    --admin <admin-user-name>
```


Replace `<adin-user-name>` with your choice. This effectively build the machine on start. I'm not sure whether the startup script will run everytime the machine starts, and I think it should not. Nevertheless, I can actually run this myself inside the VM.

Wais for 5-10 minutes until the installation of jupyterhub is completed.


## Lab configuration

Install conda/pip packages globally:
    
```
sudo -E conda install -c conda-forge gdal
sudo -E pip install there
```
    
Change default user interface to lab:

```
sudo tljh-config set user_environment.default_app jupyterlab
```

Let's Encrypt the server. First add an DNS A record of your domain name `yourhub.yourdomain.edu`, and do:

```
sudo tljh-config set https.enabled true
sudo tljh-config set https.letsencrypt.email you@example.com
sudo tljh-config add-item https.letsencrypt.domains yourhub.yourdomain.edu
```

Enable and install extensions. The web interface should work, but it didn't. I have to install on command line with sudo. I don't know if it is possible to install extensions locally for each user.

At some point, you'll want to

```
tljh-config reload
tljh-config reload proxy
```

## Lab extensions
Prerequisit:
```
conda install -c conda-forge nodejs
```

Management:
```
jupyter labextension list
# FIXME what is serverextension?
jupyter serverextension list
jupyter lab build
```

Install:

```
jupyter labextension install my-extension
jupyter labextension uninstall my-extension
jupyter labextension disable my-extension
jupyter labextension enable my-extension
```

The lab extensions installation is not smooth. That's because of permission.

In [1]:
!jupyter lab paths

Application directory:   /opt/tljh/user/share/jupyter/lab
User Settings directory: /home/jupyter-lihebi/.jupyter/lab/user-settings
Workspaces directory: /home/jupyter-lihebi/.jupyter/lab/workspaces


The Application directory is where to put extension. I should have it set locally (but ideally should still load the system one, see [this issue](https://github.com/jupyterlab/jupyterlab/issues/4064)). But according to the issue, we can at least set the

```
export JUPYTERLAB_DIR=/home/jovyan/.lab
jupyter lab build
```

- But where should we change env variable that gets loaded by jupyterlab?
- And ideally, I should have a declarative way of installing extensions.

In [11]:
!echo $HOME
!echo $JUPYTERLAB_DIR

/home/jupyter-lihebi



In [12]:
# However, this does NOT set the variable
!export JUPYTERLAB_DIR=$HOME/.lab

In [13]:
!echo $JUPYTERLAB_DIR

In [ ]:
!ls -l /opt/tljh

In [ ]:
!ls -l /

Thus, currently, I'm installing lab extensions in command line by sudo. The following extensions from (https://github.com/mauhai/awesome-jupyterlab) are interesting:

```
pip install --upgrade jupyterlab-git
jupyter lab build
jupyter labextension install @jupyterlab/toc
jupyter labextension install @jupyterlab/git
jupyter labextension install @aquirdturtle/collapsible_headings
jupyter labextension install @jupyter-widgets/jupyterlab-manager@2.0
jupyter labextension install @ijmbarr/jupyterlab_spellchecker
pip install jupyterlab_latex
jupyter labextension install @jupyterlab/latex
jupyter labextension install @lckr/jupyterlab_variableinspector
```

Some other extensions only support python. Including:
- [auto formatter](https://github.com/ryantam626/jupyterlab_code_formatter)
  - the supporting of other languages is in [this issue](https://github.com/ryantam626/jupyterlab_code_formatter/issues/24)
- lsp for jump to def https://github.com/krassowski/jupyterlab-lsp
- goto def for python https://github.com/krassowski/jupyterlab-go-to-definition

The jupyter notebook extension is completely incompatible with lab, because lab and notebook backend is completely separate `(^_^!!)`. Here's some interesting notebook extensions:

- Live Markdown Preview


# Setup IJulia and IRacket

## IJulia
Install julia first:

```
wget https://julialang-s3.julialang.org/bin/linux/x64/1.5/julia-1.5.0-linux-x86_64.tar.gz
tar zxvf julia-1.5.0-linux-x86_64.tar.gz
mv julia-1.5.0 /opt/
ln -fs /opt/julia-*/bin/julia /usr/local/bin/julia
```

Install IJulia

```
julia -e 'import Pkg; Pkg.update()'
julia -e "using Pkg; pkg\"add IJulia\"; pkg\"precompile\""
```

Optionally move kernelspec out:

```
mv "${HOME}/.local/share/jupyter/kernels/julia"* "${CONDA_DIR}/share/jupyter/kernels/"
chmod -R go+rx "${CONDA_DIR}/share/jupyter"
rm -rf "${HOME}/.local"
fix-permissions "${JULIA_PKGDIR}" "${CONDA_DIR}/share/jupyter
```

Reload the web for it to take action. No need to reload jupyter server.

## IRacket

Install racket

```
# may need to install add-apt-repository by:
sudo apt-get install software-properties-common
sudo add-apt-repository ppa:plt/racket
sudo apt-get update
sudo apt install racket
```

I'm using my fork of iracket because I'm adding something.

```
sudo apt install libzmq5
git clone https://github.com/lihebi/iracket
cd iracket && git checkout dev
raco pkg install
raco iracket install
```

# GPU Instance

I should be able to change the instance type, even add a GPU. Thus, it seems to be pretty easy to just use this instance for both CPU and GPU. If not, I should save a snapshot and duplicate instance.

But during the installation of GPU driver, the GPU may need to appear. Thus, it might be the case where I have to keep a separate instance.

UPDATE: it looks like I can install it without any problems.

```
curl -O https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-ubuntu2004.pin
sudo mv cuda-ubuntu2004.pin /etc/apt/preferences.d/cuda-repository-pin-600
sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/7fa2af80.pub
sudo add-apt-repository "deb http://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/ /"

sudo apt update
sudo apt install cuda
```